In [9]:
# -*- coding: utf-8 -*-

"Contains functions to perform dynamic FBA."

from __future__ import absolute_import

from optlang.symbolics import Zero

import numpy


def dynamic_fba(model_and_rxns, start_time, end_time, steps, solver):
    """
    Perform dynamic FBA to simulate batch growth of an organism.
    
    Dynamic FBA finds application in industrial fermentation process
    and modeling community growth of microorganisms, among others. It
    is an important tool which enhances FBA by adding reaction and
    metabolite dynamics. It can be be performed using mainly 3
    techniques: (1) Static Optimization Approach (SOA), 
    (2) Dynamic Optimization Approach, and (3) Direct Approach.
    This algorithm uses the Direct Approach which circumvents the
    inaccuracy of SOA and the Non-Linear Programming (NLP) complexity
    found in DOA. This approach employs Lexicographic Linear Programming
    to provide unique flux distribution and also solves the LP
    feasibility problem when integrating in DAE.
    
    Parameters
    ----------
    model_and_rxns: dict(model: cobra.Model, biomass: cobra.Reaction.id,
                         exchanges: tuple(cobra.Reaction.id))
        The model(s) to perform dynamic FBA on and the metabolites to track.
    start_time: float
        The time to start simulation.
    stop_time: float
        The time to end simulation.
    steps: int
        No. of time steps required for simulation.
    solver: cobra.Solver, optional
        The LP solver to use.
    
    Returns
    -------

        
    References
    ----------
    .. [1] Gomez et al.: DFBAlab: a fast and reliable MATLAB code for dynamic flux balance analysis.
           BMC Bioinformatics 2014 15:409.
    """
    try:
        model = model_and_metabolites['model']
        biomass = model_and_metabolites['biomass']
        ex_rxns = model_and_metabolites['exchanges']
    except KeyError:
        print('Incorrect arguments.')
    # TODO:
    # 1. ODE integration
    model.solver = solver
    time_range = numpy.linspace(start_time, end_time, num=steps)
    with model:
        for model in models:
            llp_sol = lexicographic_lp(model, [biomass, ex_rxns])


def lexicographic_lp(model, rxn_list):
    """
    Perform Lexicographic Linear Programming.
    
    Parameters
    ----------
    model: cobra.Model
        The model to perform lexicographic LP on.
    rxn_list: list(cobra.Reaction.id, tuple(cobra.Reaction.id)
        The list containing the reactions to be considered as objectives.
        
    Returns
    -------
    cobra.Solution
    """
    biomass_rxn_id, ex_rxn_ids = rxn_list
    
    with model:
        # LP feasibility
        obj_vars = []
        for met in model.metabolites:
            s = model.problem.Variable("s_" + met.id, lb=0)
            beta = model.problem.Variable("beta_" + met.id, lb=0)
            #
            s_equal_beta_const = model.problem.Constraint(
                s - beta,
                name="s_equal_beta_" + met.id, ub=0.0, lb=0.0)
            model.add_cons_vars([s, beta, s_equal_beta_const])
            model.constraints[met.id].set_linear_coefficients({s: 1.0, beta: -1.0})
            obj_vars.append(s)
        model.objective = model.problem.Objective(Zero, sloppy=True, direction="min")
        model.objective.set_linear_coefficients({v: 1.0 for v in obj_vars})
        model.objective_direction = "min"
        sol_feasibility = model.slim_optimize()
        feasibility_constraint = model.problem.Constraint(
            model.objective.expression,
            name="fixed_feasibility", ub=sol_feasibility, lb=sol_feasibility)
        model.add_cons_vars([feasibility_constraint])
        
        # Biomass
        model.objective = model.reactions.get_by_id(biomass_rxn_id)
        model.objective_direction = "max"
        sol_biomass = model.slim_optimize()
        biomass_constraint = model.problem.Constraint(
            model.objective.expression,
            name="fixed_biomass", ub=sol_biomass, lb=sol_biomass)
        model.add_cons_vars([biomass_constraint])
        
        # Exchanges
        for rxn_id in ex_rxn_ids:
            model.objective = model.reactions.get_by_id(rxn_id)
            model.objective_direction = "min"
            sol = model.slim_optimize()
            exchange_constraint = model.problem.Constraint(
                model.objective.expression,
                name="fixed_" + rxn_id, ub=sol, lb=sol)
            model.add_cons_vars([exchange_constraint])

        return model.optimize()

In [2]:
import cobra

In [3]:
import cobra.test

In [4]:
from optlang.symbolics import Zero

In [5]:
model = cobra.test.create_test_model('textbook')

In [10]:
first = lexicographic_lp(model, ['Biomass_Ecoli_core', ('EX_glc__D_e', 'EX_co2_e')])

In [8]:
iml1515 = cobra.io.read_sbml_model('iML1515.xml')

In [12]:
second = lexicographic_lp(iml1515, ['BIOMASS_Ec_iML1515_core_75p37M', ('EX_glc__D_e', 'EX_co2_e', 'EX_nh4_e', 'EX_pi_e', 'EX_k_e')])

In [13]:
second.loc[~numpy.isclose(second["minimum"], second["maximum"], atol=1E-09), :]

,minimum,maximum
DHORD5,0.000000e+00,1.425997e-03
FBA,5.564027e+00,6.811526e+00
DHORD2,2.886803e-01,2.901063e-01
NDPK2,8.767034e-11,3.729720e-01
NDPK3,8.767034e-11,1.587391e-01
NDPK4,8.767034e-11,2.294751e-02
NDPK1,7.316903e-11,6.928401e-01
DHAPT,1.028315e+00,2.275814e+00
F6PA,1.028315e+00,2.275814e+00
ACt2rpp,-4.564770e-03,0.000000e+00


In [11]:
first.loc[~numpy.isclose(first["minimum"], first["maximum"], atol=1E-09), :]

,minimum,maximum
